In [ ]:
import pandas as pd
from google.colab import files
import io

print("Please upload your CSV file:")
uploaded = files.upload()

# Assuming you upload only one CSV file
for filename in uploaded.keys():
  print(f'User uploaded file "{filename}"')
  # Read the CSV into a pandas DataFrame
  df = pd.read_csv(io.StringIO(uploaded[filename].decode('utf-8')))
  print(f'DataFrame loaded successfully from {filename}.')
  break # Exit loop after processing the first file

# Display the first 5 rows of the DataFrame
# print(df.head())

Please upload your CSV file:


Saving customer_shopping_behavior.csv to customer_shopping_behavior.csv
User uploaded file "customer_shopping_behavior.csv"
DataFrame loaded successfully from customer_shopping_behavior.csv.


In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
#df.describe()
df.describe(include = 'all') ## to use non numerical value for statistical analysis

In [ ]:
df.isnull().sum()

#in the output rating has missing values for rating so we can find out median rating for each category to avoid the bias and instead of setting median value for all 37 missing rating.

In [ ]:
df['Review Rating'] = df.groupby('Category')['Review Rating'].transform(lambda x: x.fillna(x.median()))

In [ ]:
# we are converting column names to snake case to make it consistent
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ', '_')
# purchase amount (usd ) - is different than other so renaming it
df = df.rename(columns={'purchase_amount_(usd)': 'purchase_amount'})

In [ ]:
df.columns

In [ ]:
# create a new column age_group to group customers in 4 age group- beneficial for marketing analysis

labels = ['Young Adult', 'Adult', 'Middle-aged','senior']
df['age_group']= pd.qcut(df['age'], q=4, labels=labels)

In [ ]:
df[['age','age_group']].head(10)

In [ ]:
#create column purchase_frequency_days - working with text makes analysis harder so converting it into numbers
#first creating a mapping for freq

frequency_mapping = {
    'Fortnightly': 14,
    'Weekly': 7,
    'Monthly': 30,
    'Quarterly': 90,
    'Bi-Weekly': 14,
    'Annually': 365,
    'Every 3 Months': 90
}

df['purchase_frequency_days'] = df['frequency_of_purchases'].map(frequency_mapping)

In [ ]:
df[['purchase_frequency_days','frequency_of_purchases']].head(10)

,purchase_frequency_days,frequency_of_purchases
0,14,Fortnightly
1,14,Fortnightly
2,7,Weekly
3,7,Weekly
4,365,Annually
5,7,Weekly
6,90,Quarterly
7,7,Weekly
8,365,Annually
9,90,Quarterly


In [ ]:
# promo code and discount applied has similar values need to check if they are redundant
#df[['discount_applied','promo_code_used']].head(10)

(df['discount_applied'] == df['promo_code_used']).all()


##output is true which means both columns has same values


np.True_

In [ ]:
df = df.drop('promo_code_used',axis = 1)

In [ ]:
df.columns

Index(['customer_id', 'age', 'gender', 'item_purchased', 'category',
       'purchase_amount', 'location', 'size', 'color', 'season',
       'review_rating', 'subscription_status', 'shipping_type',
       'discount_applied', 'previous_purchases', 'payment_method',
       'frequency_of_purchases', 'age_group', 'purchase_frequency_days'],
      dtype='object')

In [ ]:
import sqlite3
conn = sqlite3.connect("mydb.sqlite")

df.to_sql(
    name="customer_behavior",
    con=conn,
    if_exists="replace",   # 'append' to add rows
    index=False
)



3900

In [ ]:
df1 = pd.read_sql("select * from customer_behavior limit 5", conn);

In [ ]:
df1

In [ ]:
df.columns

Index(['customer_id', 'age', 'gender', 'item_purchased', 'category',
       'purchase_amount', 'location', 'size', 'color', 'season',
       'review_rating', 'subscription_status', 'shipping_type',
       'discount_applied', 'previous_purchases', 'payment_method',
       'frequency_of_purchases', 'age_group', 'purchase_frequency_days'],
      dtype='object')

In [ ]:
#Q1. What is the total revenue generated by male vs. female customers?
df2 = pd.read_sql("select gender, sum(purchase_amount) as revenue from customer_behavior group by gender",conn)
df2

,gender,revenue
0,Female,75191
1,Male,157890


In [ ]:
#Q2. Which customers used a discount but still spent more than the average purchase amount?
df3 = pd.read_sql("""
SELECT customer_id,purchase_amount
FROM customer_behavior
WHERE discount_applied = 'Yes'
  AND purchase_amount > (
      SELECT AVG(purchase_amount)
      FROM customer_behavior
  )
""", conn)

df3


,customer_id,purchase_amount
0,2,64
1,3,73
2,4,90
3,7,85
4,9,97
...,...,...
834,1667,64
835,1671,73
836,1673,73
837,1674,62


In [ ]:
#Q3. Which are the top 5 products with the highest average review rating?

df4 = pd.read_sql("""
SELECT item_purchased, round(AVG(review_rating),2) as avg_rating
FROM customer_behavior
group by item_purchased
order by avg(review_rating) desc
limit 5
 """,
conn)

df4

,item_purchased,avg_rating
0,Gloves,3.86
1,Sandals,3.84
2,Boots,3.82
3,Hat,3.80
4,Skirt,3.78


In [ ]:
#Q4. Compare the average Purchase Amounts between Standard and Express Shipping.

df5 = pd.read_sql("""
SELECT round(avg(purchase_amount),2) as avg_purchase_amount, shipping_type
FROM customer_behavior
where shipping_type in ('Standard','Express')
group by shipping_type

 """,
conn)

df5

,avg_purchase_amount,shipping_type
0,60.48,Express
1,58.46,Standard


In [ ]:
#Q5.Do subscribed customers spend more? Compare average spend and total revenue between subscribers and non-subscribers.

df6 = pd.read_sql("""
SELECT count(customer_id) AS total_customers, round(avg(purchase_amount),2) as avg_revenue,
       round(sum(purchase_amount),2) as total_revenue,subscription_status
FROM customer_behavior
group by subscription_status
order by total_revenue, avg_revenue desc

 """,
conn)

df6

,total_customers,avg_revenue,total_revenue,subscription_status
0,1053,59.49,62645.0,Yes
1,2847,59.87,170436.0,No


In [ ]:
#Q6.Which 5 products have the highest percentage of purchases with discounts applied?

df7 = pd.read_sql("""
SELECT item_purchased,
ROUND(100.0 * SUM(CASE WHEN discount_applied = 'Yes' then 1 else 0 END)/count(*),2) as discount_rate
FROM customer_behavior
where discount_applied = 'Yes'
group by item_purchased
order by discount_rate desc
limit 5;

 """,
conn)

df7

,item_purchased,discount_rate
0,Hat,50.00
1,Sneakers,49.66
2,Coat,49.07
3,Sweater,48.17
4,Pants,47.37


In [ ]:
#Q7.Segment customers into New, Returning, and Loyal based on their total number of previous purchases, and show the count of each segment.

df8 = pd.read_sql("""
with customer_type as(
SELECT customer_id,previous_purchases,
CASE
  WHEN previous_purchases = 1 THEN 'New'
  WHEN previous_purchases BETWEEN 2 AND 10 THEN 'Returning'
  ELSE 'Loyal'
  END AS Customer_Segment
FROM customer_behavior )

select customer_segment, count(*) as customer_count
from customer_type
group by customer_segment
order by customer_count desc


 """,
conn)

df8

,Customer_Segment,customer_count
0,Loyal,3116
1,Returning,701
2,New,83


In [ ]:
#Q8.What are the top 3 most purchased products within each category?

df9 = pd.read_sql("""
with product_category as(
SELECT item_purchased, category,
count(customer_id) as total_orders,
ROW_NUMBER() OVER (PARTITION BY category ORDER BY count(customer_id) desc) as product_rank
FROM customer_behavior
group by category,item_purchased
)

select product_rank, category, item_purchased, total_orders
from product_category
where product_rank <= 3;


 """,
conn)

df9

,product_rank,category,item_purchased,total_orders
0,1,Accessories,Jewelry,171
1,2,Accessories,Sunglasses,161
2,3,Accessories,Belt,161
3,1,Clothing,Pants,171
4,2,Clothing,Blouse,171
5,3,Clothing,Shirt,169
6,1,Footwear,Sandals,160
7,2,Footwear,Shoes,150
8,3,Footwear,Sneakers,145
9,1,Outerwear,Jacket,163


from matplotlib import pyplot as plt
df9['product_rank'].plot(kind='hist', bins=20, title='product_rank')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df9['total_orders'].plot(kind='hist', bins=20, title='total_orders')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
df9.groupby('category').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df9.plot(kind='scatter', x='product_rank', y='total_orders', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
df9['total_orders'].plot(kind='line', figsize=(8, 4), title='total_orders')
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(df9['category'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(df9, x='product_rank', y='category', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(df9['category'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(df9, x='total_orders', y='category', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
#Q9.Are customers who are repeat buyers (more than 5 previous purchases) also likely to subscribe?

df10 = pd.read_sql("""
SELECT subscription_status, count(customer_id) as repeat_buyers
FROM customer_behavior
WHERE previous_purchases > 5
GROUP BY subscription_status

 """,
conn)

df10

,subscription_status,repeat_buyers
0,No,2518
1,Yes,958


In [ ]:
#Q10.What is the revenue contribution of each age group?

df11 = pd.read_sql("""
SELECT sum(purchase_amount) as total_revenue, age_group
FROM customer_behavior
GROUP BY age_group
order by total_revenue desc

 """,
conn)

df11

,total_revenue,age_group
0,62143,Young Adult
1,59197,Middle-aged
2,55978,Adult
3,55763,senior


In [ ]:
df.to_csv("output.csv", index=False)